<h1>Wirecell Clustering v0_xx_0 (sbndcode v0x_xx_xx)</h1>

GitHub commands: http://localhost:8888/notebooks/OneDrive%2FDocumentos%2Fbackup-2023-xx-xx%2FMeetings%2FWireCell%2FMisc%2FMisc.ipynb

<h5>Code locations (make sure to push changes to personal github frequently)

<h5>Setup Apptainer

In [ ]:
/cvmfs/oasis.opensciencegrid.org/mis/apptainer/current/bin/apptainer shell --shell=/bin/bash -B /cvmfs,/exp,/nashome,/pnfs/sbn,/pnfs/sbnd,/opt,/run/user,/etc/hostname,/etc/hosts,/etc/krb5.conf --ipc --pid /cvmfs/singularity.opensciencegrid.org/fermilab/fnal-dev-sl7:latest

<h5>Setup sbndcode

In [ ]:
source /cvmfs/sbnd.opensciencegrid.org/products/sbnd/setup_sbnd.sh
setup sbndcode v09_91_02 -q e26:prof

<h5>Setup larwirecell

In [ ]:
cd /exp/sbnd/data/users/ebatista/wirecell-clustering
source larsoft_v09_91_02_e26_prof/localProducts_larsoft_v09_91_02_e26_prof/setup
mrbsetenv
mrbslp

<h5>Setup wirecell (configuration)

In [ ]:
export LD_LIBRARY_PATH=/exp/sbnd/app/users/yuhw/opt/lib/:$LD_LIBRARY_PATH
export WIRECELL_PATH=/exp/sbnd/app/users/yuhw/wct-cfg/cfg:$WIRECELL_PATH
export WIRECELL_PATH=/exp/sbnd/app/users/yuhw/wire-cell-data:$WIRECELL_PATH


<h5>Compile wirecell (if needed)

In [ ]:
# push changes to github first ...
# cd /exp/sbnd/data/users/ebatista/wirecell-clustering/wire-cell-toolkit
# git status
# git add <files>
# git... 

# ... then compile
cd /exp/sbnd/data/users/ebatista/wirecell-clustering/wire-cell-toolkit
./wcb configure \
--prefix=/exp/sbnd/data/users/ebatista/wirecell-clustering/wire-cell-toolkit/install \
--with-spdlog=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2 \
--with-spdlog-include=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e20-prof/include \
--with-spdlog-lib=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e20-prof/lib64 \
--with-jsoncpp=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a \
--with-jsoncpp-include=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e20-prof/include \
--with-jsoncpp-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e20-prof/lib \
--with-eigen=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v3_4_0 \
--with-eigen-include=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v3_4_0/include/eigen3 \
--with-jsonnet-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/lib \
--with-jsonnet-include=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/include \
--boost-includes=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_80_0/Linux64bit+3.10-2.17-e20-prof/include \
--boost-libs=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_80_0/Linux64bit+3.10-2.17-e20-prof/lib
./wcb -p -v --notests install
cd -

<h5>Setup wirecell (configuration)

In [ ]:
export WIRECELL_PATH=/exp/sbnd/app/users/ebatista/test_sbndcode_v09_83_01_e26_prof/wire-cell-cfg:/exp/sbnd/app/users/ebatista/test_sbndcode_v09_83_01_e26_prof/wire-cell-data:$WIRECELL_PATH
export LD_LIBRARY_PATH=/exp/sbnd/data/users/ebatista/wirecell-clustering/wire-cell-toolkit/install/lib64:$LD_LIBRARY_PATH

<h5>Run clustering

In [ ]:
Run clustering (wire-cell executable only available on gpvm for now):

cd /exp/sbnd/data/users/ebatista/wirecell-clustering
source /exp/sbnd/app/users/ebatista/venv/bin/activate
time lar -n 1 -c wcls-sig-to-img_v4.fcl -s /exp/sbnd/app/users/yuhw/wcp-porting-img/sbnd/input-moon.root -o tmp.root >& log

In [ ]:
Visualize graph:

jsonnet -J \
/exp/sbnd/app/users/ebatista/test_sbndcode_v09_83_01_e26_prof/wire-cell-cfg \
/exp/sbnd/app/users/ebatista/img-clustering/wcp-porting-img/wct-uboone-clustering-copy.jsonnet \
-o wct-uboone-clustering-copy.json

wirecell-pgraph dotify --jpath -1 --no-params \
wct-uboone-clustering-copy.json \
wct-uboone-clustering-copy.pdf

<h3>Dec 19, 2024

[try this] https://github.com/HaiwangYu/wcp-porting-img/tree/main/sbnd
[input files] /exp/sbnd/app/users/yuhw/wcp-porting-img/sbnd/
[setup] https://github.com/HaiwangYu/wcp-porting-img/blob/main/sbnd/setup.sh
[setup here as well] /exp/sbnd/app/users/yuhw/wcp-porting-img/sbnd/setup.sh

<h4>------------------TODO------------------

<h5>Dec 20, 2024</h5>
0-0.6 GeV low energy range
0.6-1.5 medium
1.5-3.0 high
 
10 to 20 events of:
0. Cosmics 
1. NumuCC
2. nueCC
3. NC

Create machinery from gen to 3D bee display to save JSON info
 
1. Handscan
2. Quantitative
3. Completeness/Efficiciency

Check how to save MC info to make 3D view in bee display 